In [33]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##### Copyright 2018 The TensorFlow Authors.

Keras - высокоуровневый API для создания моделей глубокого обучения. Он используется для быстрого создания прототипов, сложных исследований, а также для создания приложений.

# Импортируем tf.keras
tf.keras - это реализация спецификации Keras API  в TensorFlow. Это высокоуровневый API для построения моделей глубокого обучения.

In [11]:
import tensorflow as tf
from tensorflow.python.keras import layers 

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


# Построим простую модель
## Последовательная модель

В Keras мы используем слои для построения моделей. Обычно модель -- это граф, состоящий из нескольких словев. Самый распространенный тип модели это стэк идущих друг за другом слоев - последовательная модель - `` tf.keras.Sequential `` .

Для начала построим простую полносвязную сеть (или многослойный персептрон):


In [12]:
model = tf.keras.Sequential()
# Добавим в нашу модель слой Dense из 64 блоков:
model.add(layers.Dense(64, activation='relu'))
# И еще один:
model.add(layers.Dense(64, activation='relu'))
# Также добавим слой softmax с 10 выводящими блоками:
model.add(layers.Dense(10, activation='softmax'))

## Конфигурация слоев

Существует много разных слоев `tf.keras.layers` с общими параметрами конфигурации:

* `activation`: Устанавливает функцию активации для данного слоя. Этот параметр должен указываться в имени встроенной функции или использоваться как вызываемый объект. По умолчанию активация не используется
* `kernel_initializer` и `bias_initializer`: Инициализация, которая создает веса данного слоя (ядро kernel и смещение bias). Этот параметр используется как имя или вызываемый объект. По умолчанию используется инициализатор `"Glorot uniform"`
* `kernel_regularizer` и `bias_regularizer`: Регуляризация, которая применяется к весам слоя (ядро kernel и смещение bias),  например L1 или L2. По умолчанию не используется

Следующий код используется для построения `tf.keras.layers.Dense` с настройкой конфигурации каждого слоя:

In [14]:
# Sigmoid layer
layers.Dense(64, activation='sigmoid')
# or
layers.Dense(64, activation=tf.sigmoid)

# Линейный слой с регуляризацией L1 с коэффицентом 0.01 примененная к ядру матрицы:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# Линейный слой с регуляризацией L2 с коэффицентом 0.01 примененная к вектору смещения (bias):
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# Линейный слой с ядром, инициализированным в случайной прямоугольный матрице:
layers.Dense(64, kernel_initializer='orthogonal')

# Линейный слой с вектором смещения, инициализированным с коэффицентом 2:
layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

## Обучение и оценка

### Настроим конфигурацию обучения

После того как модель построена давай обозначим процесс обучения с помощью метода `compile`:

In [16]:
model = tf.keras.Sequential([
# Добавим в нашу модель слой `Dense` из 64 блоков:
layers.Dense(64, activation='relu', input_shape=(32,)),
# И еще один:
layers.Dense(64, activation='relu'),
# Также добавим слой softmax с 10 выводящими блоками:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


У `tf.keras.Model.compile` есть три важных аргумента:

* `optimizer`: Этот объект определяет процедуру обучения. Мы будем использовать оптимизаторы из модуля `tf.train`, например такие как `tf.train.AdamOptimizer`, `tf.train.RMSPropOptimizer` и `tf.train.GradientDescentOptimizer`
* `loss`: Эта функция минимизации для решения задач оптимизации. Самыми распространенными выборами этого аргумента являются среднеквадратическое отклонение (`mse`), `categorical_crossentropy`, и `binary_crossentropy`. Функции потерь обозначены по имени или используются как вызываемые объекты из модуля `tf.keras.losses`
* `metrics`: Используются для наблюдения за процессом обучения. Используются как строки или вызываемые объекты модуля `tf.keras.metrics`

Ниже пример конфигурации модели для обучения:

In [17]:
# Настраиваем регрессию модели, используем среднеквадратическое отклонение
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # среднеквадратическое отклонение
              metrics=['mae'])  # средняя абсолютная ошибка

# Настраиваем модель для классификации по категориям
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

Instructions for updating:
Use tf.cast instead.


### Используем данные NumPy

При работе с небольшими датасетами мы будем использовать загружаемые в память массивы данных [NumPy](https://www.numpy.org/){:.external} для обучения и оценки моделей. Модель будет "подстраиваться" под тренировочные данные при помощи метода `fit`:

In [18]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Instructions for updating:
Use tf.cast instead.


Epoch 1/10


  32/1000 [..............................] - ETA: 32s - loss: 12.1696 - categorical_accuracy: 0.1562

 256/1000 [======>.......................] - ETA: 3s - loss: 11.7238 - categorical_accuracy: 0.0898 

1000/1000 [==============================] - 1s 1ms/sample - loss: 11.6093 - categorical_accuracy: 0.0970


...

1000/1000 [==============================] - 0s 69us/sample - loss: 11.5472 - categorical_accuracy: 0.1100


Epoch 10/10
  32/1000 [..............................] - ETA: 0s - loss: 11.6494 - categorical_accuracy: 0.1562



 864/1000 [========================>.....] - ETA: 0s - loss: 11.5911 - categorical_accuracy: 0.1227

1000/1000 [==============================] - 0s 67us/sample - loss: 11.5407 - categorical_accuracy: 0.1170


У `tf.keras.Model.fit` есть три важных аргумента:

* `epochs`: Процесс обучения структурирован по *эпохам*. Эпоха означает один проход по всему набору входных данных, который происходит небольшими порциями в батчах
* `batch_size`: Когда мы используем массивы NumPy модель делит данные на небольшие батчи и затем выполняет указанное количество проходов. Это число определяет размер батча. Помни, что последний батч может быть меньше если общее количество образцов данных не делится на размер батча
* `validation_data`: Когда мы строим прототип модели, мы хотим легко наблюдать за ее точностью на проверочных данных. При помощи данного аргумента - обычно кортеж или метка - модель будет отображать потери и статистику в режиме выводов inference для прошедших через модель данных в конце каждой эпохи
  
Вот пример использования `validation_data`:

In [19]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 0s - loss: 11.3244 - categorical_accuracy: 0.1250

 672/1000 [===================>..........] - ETA: 0s - loss: 11.5202 - categorical_accuracy: 0.1116

1000/1000 [==============================] - 0s 186us/sample - loss: 11.5906 - categorical_accuracy: 0.1000 - val_loss: 11.6959 - val_categorical_accuracy: 0.1000


Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 11.2457 - categorical_accuracy: 0.1875



...

1000/1000 [==============================] - 0s 76us/sample - loss: 11.5839 - categorical_accuracy: 0.1160 - val_loss: 11.6847 - val_categorical_accuracy: 0.1500


Epoch 10/10
  32/1000 [..............................] - ETA: 0s - loss: 11.4015 - categorical_accuracy: 0.0625

 864/1000 [========================>.....] - ETA: 0s - loss: 11.6073 - categorical_accuracy: 0.0961



1000/1000 [==============================] - 0s 78us/sample - loss: 11.5834 - categorical_accuracy: 0.0980 - val_loss: 11.6948 - val_categorical_accuracy: 0.0800


### Используем датасеты tf.data

Чтобы обучать модель на больших датасетах или на нескольких устройствах мы можем воспользоваться [Datasets API](./datasets.md). Просто добавь `tf.data.Dataset` к методу `fit`:

In [20]:
# Инициализируем пробную инстанцию датасета:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

# Не забудь указать количество шагов в каждой эпохе `steps_per_epoch` при использовании метода `fit`
model.fit(dataset, epochs=10, steps_per_epoch=30)

Epoch 1/10


 1/30 [>.............................] - ETA: 15s - loss: 11.6878 - categorical_accuracy: 0.1562

26/30 [=========================>....] - ETA: 0s - loss: 11.5741 - categorical_accuracy: 0.1178 

30/30 [==============================] - 1s 20ms/step - loss: 11.5892 - categorical_accuracy: 0.1177


Epoch 2/10
 1/30 [>.............................] - ETA: 0s - loss: 11.3693 - categorical_accuracy: 0.1250

...

30/30 [==============================] - 0s 3ms/step - loss: 11.5944 - categorical_accuracy: 0.1474


Epoch 10/10
 1/30 [>.............................] - ETA: 0s - loss: 11.1458 - categorical_accuracy: 0.2188

24/30 [=======================>......] - ETA: 0s - loss: 11.6295 - categorical_accuracy: 0.1398



30/30 [==============================] - 0s 2ms/step - loss: 11.5718 - categorical_accuracy: 0.1368


Таким образом, метод `fit` использует аргумент `steps_per_epoch` - это количество шагов обучения, которые модель должна сделать прежде, чем перейти на следующую эпоху. Поскольку `Dataset` принимает батчи данных, то в этом примере кода нам не требуется указывать размер батча в `batch_size`.

Также `dataset` можно использовать для проверки точности модели:

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

Epoch 1/10


 1/30 [>.............................] - ETA: 9s - loss: 11.6666 - categorical_accuracy: 0.1875

22/30 [=====================>........] - ETA: 0s - loss: 11.5207 - categorical_accuracy: 0.1491



30/30 [==============================] - 0s 17ms/step - loss: 11.5650 - categorical_accuracy: 0.1437 - val_loss: 11.7205 - val_categorical_accuracy: 0.0729


...

30/30 [==============================] - 0s 2ms/step - loss: 11.5640 - categorical_accuracy: 0.1549 - val_loss: 11.7624 - val_categorical_accuracy: 0.1250


Epoch 10/10
 1/30 [>.............................] - ETA: 0s - loss: 11.1197 - categorical_accuracy: 0.1562

22/30 [=====================>........] - ETA: 0s - loss: 11.5903 - categorical_accuracy: 0.1529



30/30 [==============================] - 0s 3ms/step - loss: 11.5389 - categorical_accuracy: 0.1667 - val_loss: 11.7918 - val_categorical_accuracy: 0.1324


### Оценка и предсказание

Методы `tf.keras.Model.evaluate` и `tf.keras.Model.predict` могут использовать данные NumPy и `tf.data.Dataset`.

Используй следующий пример кода для оценки потерь и других показателей предсказаний на использованных данных:

In [22]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

model.evaluate(dataset, steps=30)

  32/1000 [..............................] - ETA: 6s - loss: 11.7958 - categorical_accuracy: 0.1250

1000/1000 [==============================] - 0s 272us/sample - loss: 11.6869 - categorical_accuracy: 0.1070


 1/30 [>.............................] - ETA: 5s - loss: 11.6723 - categorical_accuracy: 0.1562

30/30 [==============================] - 0s 8ms/step - loss: 11.5428 - categorical_accuracy: 0.1479


[11.542752583821615, 0.14791666]

Для *предсказания* вывода последнего слоя как массив NumPy, используй следующий код:

In [24]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


## Построение сложных моделей
### Функциональный API

Последовательная модель `tf.keras.Sequential` представляет из себя обычное наложение или стек слоев, которые не могут представлять произвольные модели. Используй
[функциональный API Keras ](https://keras.io/getting-started/functional-api-guide/){:.external} для построения комплексных топологий моделей, например таких как:

* Модели с множественными входами данных
* Модели с множественными выводами данных
* Модели с общими слоями, где один и тот же слой вызывается несколько раз
* Модели с непоследовательным потоком данных, например где есть остаточные связи

Построение модели при помощи функционального API выглядит следующим образом:

1. Слой вызывается и возвращает тензор
2. Для определения `tf.keras.Model` используем входные и выводящие тензоры
3. Модель обучается точно так же, как и `Sequential`

Следующий пример показывает как использовать функциональный API для построения простой полносвязной сети:

In [26]:
# Возвращает тензор-"заполнитель", который мы используем в качестве примера
inputs = tf.keras.Input(shape=(32,)) 

# Вызываемый на тензор слой, возвращает тензор
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

Обозначим вход и вывод данных нашей модели:

In [27]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# При помощи метода `compile` настраиваем конфигурацию обучения
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Тренируем модель в течение 5 эпох
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5


  32/1000 [..............................] - ETA: 6s - loss: 11.4371 - acc: 0.1250

 608/1000 [=================>............] - ETA: 0s - loss: 11.7755 - acc: 0.0757

1000/1000 [==============================] - 0s 311us/sample - loss: 11.7486 - acc: 0.0710


Epoch 2/5
  32/1000 [..............................] - ETA: 0s - loss: 11.8786 - acc: 0.1250

...

 928/1000 [==========================>...] - ETA: 0s - loss: 11.6351 - acc: 0.1034

1000/1000 [==============================] - 0s 61us/sample - loss: 11.6368 - acc: 0.1030


Epoch 5/5
  32/1000 [..............................] - ETA: 0s - loss: 11.6972 - acc: 0.0625

 800/1000 [=======================>......] - ETA: 0s - loss: 11.6451 - acc: 0.1175

1000/1000 [==============================] - 0s 74us/sample - loss: 11.6287 - acc: 0.1140


### Создание подклассов моделей

Также ты можешь создать модель с нуля при помощи подклассов в `tf.keras.Model` и определения собственных прямых проходов. Создавай слои в методе `__init__` и установи их как атрибуты класса. Прямой проход должен быть указан в методе `call`.

Создание подклассов моделей особенно полезно, когда активирован [eager execution](./eager.md), так как прямой проход в этом случае всегда будет записан.

Ключевой момент: всегда используй правильный API для решения конкретной задачи. Поскольку создание подклассов модели предполагает определенную гибкость, эта гибкость осложняет определение структуры и может повлечь больше ошибок у пользователя. Если возможно, то всегда старайся использовать функциональный API.

Смотри следующий пример кода, в котором мы будем использовать подклассы `tf.keras.Model` и собственный прямой проход:

In [28]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # Здесь определим слои
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # А здесь укажем прямой проход,
    # используя указанные выше слои (в `__init__`).
    x = self.dense_1(inputs)
    return self.dense_2(x)

  def compute_output_shape(self, input_shape):
    # Если ты хочешь использовать подклассы модели 
    # в функциональном стиле, тогда эта функция будет переписана 
    # во время запуска кода. В остальных случаях этот метод необязателен.
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.num_classes
    return tf.TensorShape(shape)

Укажем класс новой модели:

In [29]:
model = MyModel(num_classes=10)

# При помощи метода `compile` настраиваем конфигурацию обучения
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучаем в течение 5 эпох
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5


  32/1000 [..............................] - ETA: 7s - loss: 12.1450 - acc: 0.2188

 768/1000 [======================>.......] - ETA: 0s - loss: 11.7675 - acc: 0.0911

1000/1000 [==============================] - 0s 308us/sample - loss: 11.7542 - acc: 0.0950


Epoch 2/5
  32/1000 [..............................] - ETA: 0s - loss: 11.5600 - acc: 0.0938

...


  32/1000 [..............................] - ETA: 0s - loss: 12.1142 - acc: 0.0625



 736/1000 [=====================>........] - ETA: 0s - loss: 11.6403 - acc: 0.1087



1000/1000 [==============================] - 0s 85us/sample - loss: 11.6364 - acc: 0.1100


### Создание собственных слоев

Чтобы создать свой собственный слой при помощи подклассов `tf.keras.layers.Layer` нам потребуются следующие методы:

* `build`: Создает веса слоя. Чтобы добавить веса в модель используй метод `add_weight`
* `call`: Определяет прямой проход
* `compute_output_shape`: Определяет как вычислить выводящую форму слоя для данной входной формы
* Также слой можно сериализировать при помощи метода `get_config` и метода класса `from_config`

Вот пример использования нового слоя, в котором мы использовали `matmul` входа с матрицей ядра `kernel`:

In [30]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # Создаем обучаемую переменную весов для этого слоя
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Обязательно вызови метод `build` в конце
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

Теперь создадим модель, используя наш новый слой:

In [31]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# При помощи метода `compile` настраиваем конфигурацию обучения
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучаем в течение 5 эпох
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
  32/1000 [..............................] - ETA: 4s - loss: 11.7826 - acc: 0.0625

 960/1000 [===========================>..] - ETA: 0s - loss: 11.6449 - acc: 0.1073

1000/1000 [==============================] - 0s 211us/sample - loss: 11.6374 - acc: 0.1090


Epoch 2/5
  32/1000 [..............................] - ETA: 0s - loss: 11.3668 - acc: 0.1562

 768/1000 [======================>.......] - ETA: 0s - loss: 11.6600 - acc: 0.1289

...

Epoch 5/5


  32/1000 [..............................] - ETA: 0s - loss: 11.5270 - acc: 0.0938



 768/1000 [======================>.......] - ETA: 0s - loss: 11.6138 - acc: 0.0820

1000/1000 [==============================] - 0s 70us/sample - loss: 11.6249 - acc: 0.0930


## Функции обратного вызова

Функция обратного вызова `callback` - это объект, который передается модели для обработки и расширения ее поведения во время обучения. Ты можешь написать свою собственную функцию callback, или использовать готовые `tf.keras.callbacks` в которые входят:

* `tf.keras.callbacks.ModelCheckpoint`: Сохраняет контрольные точки твоей модели через заданный интервал
* `tf.keras.callbacks.LearningRateScheduler`: Замедляет темп обучения `learning rate` для получения лучших результатов точности
* `tf.keras.callbacks.EarlyStopping`: Останавливает обучение как только точность перестает увеличиваться
* `tf.keras.callbacks.TensorBoard`: Следит за поведением модели при помощи [TensorBoard](./summaries_and_tensorboard.md)

Чтобы использовать `tf.keras.callbacks.Callback` просто передай его в метод `fit` своей модели:

In [34]:
callbacks = [
  # Прерывает обучение если потери при проверке `val_loss` перестают 
  # уменьшаться после 2 эпох
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Записываем логи TensorBoard в папку `./logs`
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples


Epoch 1/5
  32/1000 [..............................] - ETA: 0s - loss: 11.0473 - acc: 0.0938

 736/1000 [=====================>........] - ETA: 0s - loss: 11.5777 - acc: 0.0965

1000/1000 [==============================] - 0s 483us/sample - loss: 11.6227 - acc: 0.1030 - val_loss: 11.6975 - val_acc: 0.0600


Epoch 2/5
  32/1000 [..............................] - ETA: 0s - loss: 12.0239 - acc: 0.0938

...

1000/1000 [==============================] - 0s 120us/sample - loss: 11.6212 - acc: 0.1110 - val_loss: 11.7061 - val_acc: 0.0800


Epoch 3/5
  32/1000 [..............................] - ETA: 0s - loss: 12.0534 - acc: 0.1875



 512/1000 [==============>...............] - ETA: 0s - loss: 11.6634 - acc: 0.1211

1000/1000 [==============================] - 0s 152us/sample - loss: 11.6191 - acc: 0.1140 - val_loss: 11.7001 - val_acc: 0.0500


## Сохранение и загрузка
### Сохраняем веса

Ты можешь сохранять и загружать веса модели при помощи `tf.keras.Model.save_weights`:

In [35]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
# Сохраняем веса в контрольную точку формата TensorFlow
model.save_weights('./weights/my_model')

# Восстанавливаем состояние модели,
# требуется использование модели с точно такой же архитектурой
model.load_weights('./weights/my_model')

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


По умолчанию, веса модели сохраняются в формате контрольой точки TensorFlow. Веса также могут быть сохранены в формате Keras HDF5, который является стандартным в бэкенд структуре Keras:

In [37]:
# Сохраняем веса в файл HDF5 
model.save_weights('my_model.h5', save_format='h5')

# Загружаем текущее состояние модели
model.load_weights('my_model.h5')

### Сохраняем конфигурацию

Конфигурация модели также может быть сохранена: такой метод сериализирует архитектуру модели без сохранения весов.

Сохраненная конфигурация можеть быть загружена и инициализирована как оригинальная модель, даже без кода изначальной модели.

Keras поддерживает форматы сериализации данных JSON и YAML:

In [38]:
# Сериализация модели в формат JSON
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_7", "layers": [{"class_name": "Dense", "config": {"name": "dense_21", "trainable": true, "dtype": null, "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_22", "trainable": true, "dtype": null, "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "ke

In [39]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense

Давай воссоздадим только что инициализированную модель из JSON:

In [40]:
fresh_model = tf.keras.models.model_from_json(json_string)

Важно: модели с подклассами не могут быть сериализированы, потому что их архитектура определяется кодом Python в методе `call`.

### Сохраняем модель полностью

Мы можем сохранить модель целиком в один файл, который будет содержать веса, конфигурацию модели и даже настройки оптимизатора. Это позволяет сохранить модель как контрольную точку и продолжить обучение позже - ровно с того же момента и без доступа к исходному коду.

In [41]:
# Создаем простую модель
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

# Сохраняем модель целиком в один файл формата HDF5
model.save('my_model.h5')

# Восстаналиваем ту же самую модель, включая веса и оптимизатор
model = tf.keras.models.load_model('my_model.h5')

Epoch 1/5


  32/1000 [..............................] - ETA: 8s - loss: 11.2800 - acc: 0.1250

 736/1000 [=====================>........] - ETA: 0s - loss: 11.6271 - acc: 0.0992

1000/1000 [==============================] - 0s 362us/sample - loss: 11.6365 - acc: 0.1100


Epoch 2/5
  32/1000 [..............................] - ETA: 0s - loss: 11.6781 - acc: 0.1250

...

 544/1000 [===============>..............] - ETA: 0s - loss: 11.6029 - acc: 0.1268

1000/1000 [==============================] - 0s 101us/sample - loss: 11.6230 - acc: 0.1230


Epoch 5/5
  32/1000 [..............................] - ETA: 0s - loss: 11.2427 - acc: 0.0938

 640/1000 [==================>...........] - ETA: 0s - loss: 11.6971 - acc: 0.1125

1000/1000 [==============================] - 0s 90us/sample - loss: 11.6227 - acc: 0.1160


### Алгоритмы оценки Estimators


API [Estimators](./estimators.md) используется для обучения моделей в распределенном окружении. Он необходим для решения задач распределенного обучения на больших датасетах например для экспорта модели в продакшен.

`tf.keras.Model` может обучаться с API `tf.estimator` посредством конвертации модели в объект `tf.estimator.Estimator` при помощи `tf.keras.estimator.model_to_estimator`. Читай больше в статье [Создание Estimators из моделей Keras](./estimators.md#creating_estimators_from_keras_models).

In [42]:
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\denkl\\AppData\\Local\\Temp\\tmpefcgxq2j', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000236A9468630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Совет: используй [eager execution](https://www.tensorflow.org/guide/eager#build_a_model) для отладки
[функций входа Estimator](./premade_estimators.md#create_input_functions) и проверки данных.

### Обучение на нескольких GPU

Модели `tf.keras` могут обучаться на нескольких GPU при помощи
`tf.contrib.distribute.DistributionStrategy`. Этот API обеспечивает распределенное обучение на нескольких GPU почти без изменений основного кода модели.

В настоящее время `tf.contrib.distribute.MirroredStrategy` является единственной поддерживаемой стратегией распределенного обучения. `MirroredStrategy` выполняет внутриграфную репликацию с синхронным обучением, используя функцию all-reduce на одном устройстве. Чтобы использовать `DistributionStrategy` вместе с Keras, конвертируй модель `tf.keras.Model` в `tf.estimator.Estimator` при помощи `tf.keras.estimator.model_to_estimator`, а затем обучи получившийся estimator.

В следующем примере мы посмотрим как распределить `tf.keras.Model` на нескольких GPU на одном устройстве.

Для начала определим простую модель:

In [44]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


Затем определим функцию *загрузки и обработки данных*. Функция `Input_fn` возвращает объект `tf.data.Dataset`, который используется для распределения данных на нескольких устройствах, где каждый GPU обрабатывает свой входящий батч.

In [45]:
def input_fn():
  x = np.random.random((1024, 10))
  y = np.random.randint(2, size=(1024, 1))
  x = tf.cast(x, tf.float32)
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.repeat(10)
  dataset = dataset.batch(32)
  return dataset

Далее, создадим `tf.estimator.RunConfig` и передадим аргумент `train_distribute` 
к  `tf.contrib.distribute.MirroredStrategy`. При создании
`MirroredStrategy` ты можешь определить список устройств или передать аргумент `num_gpus` с заданным количеством GPU для обучения. По умолчанию используются все доступные GPU как в следующем примере:

In [46]:
strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



INFO:tensorflow:Initializing RunConfig with distribution strategies.


INFO:tensorflow:Not using Distribute Coordinator.


Конвертируем модель Keras в `tf.estimator.Estimator`:


In [50]:
keras_estimator = tf.keras.estimator.model_to_estimator(
  keras_model=model,
  config=config,
  model_dir='/tmp/model_dir')

Наконец, обучаем `Estimator`, передав аргументы `input_fn` и `steps`

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=10)